In [64]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai.embeddings import OpenAIEmbeddings
# pip install openpyxl
import pandas as pd
import numpy as np
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
import time

In [28]:
# This can be a separate script 'utils.py'
def cosine_similarity(a: list, b: list) -> float:
    a = np.array(a) if isinstance(a, list) else a
    b = np.array(b) if isinstance(b, list) else b
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [58]:
# Instantiate an embedding model
embedding_model = OpenAIEmbeddings()

In [60]:
# Load the data (and embed it)
df = pd.read_excel("myData.xlsx")
df['embedding'] = df['Words'].apply(lambda x: embedding_model.embed_query(x))
df

,Words,embedding
0,Elephant,"[-0.018824968487024307, -0.008682483807206154,..."
1,Lion,"[-0.0015009930357336998, -0.010024921968579292..."
2,Tiger,"[-0.013500549830496311, -0.009651594795286655,..."
3,Dog,"[-0.0008935772348195314, -0.015069474466145039..."
4,Cricket,"[0.003957032226026058, -0.007208024617284536, ..."
5,Football,"[0.003506750101223588, -0.011377162300050259, ..."
6,Basketball,"[-0.012911115773022175, -0.013261756859719753,..."
7,Apple,"[0.014532957226037979, -0.003988702315837145, ..."
8,Orange,"[0.02082180231809616, -0.02937670238316059, 2...."
9,Banana,"[-0.01310732588171959, -0.020157383754849434, ..."


In [61]:
# Try the search
our_text = "Cat"
text_embedding = embedding_model.embed_query(our_text)
df['similarity_score'] = df['embedding'].apply(lambda x: cosine_similarity(x, text_embedding))
df = df.sort_values('similarity_score', ascending=False)
df

,Words,embedding,similarity_score
3,Dog,"[-0.0008935772348195314, -0.015069474466145039...",0.878559
2,Tiger,"[-0.013500549830496311, -0.009651594795286655,...",0.846970
1,Lion,"[-0.0015009930357336998, -0.010024921968579292...",0.840089
7,Apple,"[0.014532957226037979, -0.003988702315837145, ...",0.833735
0,Elephant,"[-0.018824968487024307, -0.008682483807206154,...",0.819722
8,Orange,"[0.02082180231809616, -0.02937670238316059, 2....",0.811868
9,Banana,"[-0.01310732588171959, -0.020157383754849434, ...",0.806824
5,Football,"[0.003506750101223588, -0.011377162300050259, ...",0.805747
4,Cricket,"[0.003957032226026058, -0.007208024617284536, ...",0.795493
6,Basketball,"[-0.012911115773022175, -0.013261756859719753,...",0.784980


In [70]:
# Using FAISS for this
loader = CSVLoader(file_path="myData.csv", csv_args={'delimiter': ',', 'quotechar': '"', 'fieldnames': ['Words']})
data = loader.load()
db = FAISS.from_documents(data, embedding_model)
db.similarity_search_with_score("Mango")[0]

(Document(page_content='Words: Banana', metadata={'source': 'myData.csv', 'row': 10}),
 0.30739564)